# Detection of Fraud Reviews: Modeling and Evaluation

In [1]:
import pandas as pd

## Reading Fake Reviews datasets

In [2]:
# Path to your CSV file
fake_reviews_path = '/Users/ShanShan/Fake-Reviews-Detection/Dataset/fake reviews dataset.csv'

# Read the CSV file
fake_reviews_df = pd.read_csv(fake_reviews_path)

In [3]:
fake_reviews_df.shape

(40432, 4)

In [4]:
fake_reviews_df.columns

Index(['category', 'rating', 'label', 'text_'], dtype='object')

In [5]:
# Display the first few rows
fake_reviews_df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


## Reading Yelp dataset

In [6]:
# Path to your Excel file
yelp_path = '/Users/ShanShan/Fake-Reviews-Detection/Dataset/Yelp Labelled Review Dataset with Sentiments and Features.xlsx'

# Read the Excel file
yelp_df = pd.read_excel(yelp_path, engine='openpyxl')

In [7]:
yelp_df.shape

(355210, 8)

In [8]:
yelp_df.columns

Index(['User_id', 'Product_id', 'Rating', 'Date', 'Review',
       'Spam(1) and Not Spam(0)', 'Sentiment', 'Features'],
      dtype='object')

In [9]:
# Display the first few rows
yelp_df.head()

,User_id,Product_id,Rating,Date,Review,Spam(1) and Not Spam(0),Sentiment,Features
0,923,0,3,2014-01-30,The food at snack is a selection of popular Gr...,1,Positive,"['appetizer tray', 'greek salad', 'main courses']"
1,924,0,3,2011-05-05,This little place in Soho is wonderful. I had ...,1,Positive,"['little place', 'soho', 'lamb sandwich', 'soh..."
2,925,0,4,2011-12-30,ordered lunch for 15 from Snack last Friday. Ã...,1,Positive,"['snack', 'regular company lunch list']"
3,926,0,4,2012-10-04,This is a beautiful quaint little restaurant o...,1,Positive,"['beautiful quaint', 'pretty street', 'great p..."
4,927,0,4,2014-02-06,Snack is great place for a Ã‚Â casual sit down...,1,Positive,"['snack', 'great place', 'Ã¢ casual', 'cold wi..."


In [ ]:
Index(['category', 'rating', 'label', 'text_'], dtype='object')

Index(['User_id', 'Product_id', 'Rating', 'Date', 'Review',
       'Spam(1) and Not Spam(0)', 'Sentiment', 'Features'],
      dtype='object')